# Supply Chain Delay Prediction System

This notebook demonstrates a complete machine learning pipeline for predicting supply chain delays using ensemble methods and advanced feature engineering.

## 1. Import Required Libraries

Import necessary libraries for data manipulation, model building, and visualization.

In [ ]:
import sys
sys.path.insert(0, '/workspaces/project-defense/supply_chain_prediction')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings

warnings.filterwarnings('ignore')

# Import our custom modules
from data_loader import SupplyChainDataGenerator
from feature_engineering import FeatureEngineer
from models import DelayPredictionModel, EnsembleDelayPredictor
from evaluation import ModelEvaluator, DelayAnalyzer
from visualization import PredictionVisualizer

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("All libraries imported successfully!")

## 2. Load and Explore Supply Chain Data

Generate synthetic supply chain data and perform exploratory data analysis.

In [ ]:
# Generate synthetic supply chain data
generator = SupplyChainDataGenerator(random_state=42)
df = generator.generate_dataset(n_samples=1000)

print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

print("\n" + "="*80)
print("DATA SUMMARY STATISTICS")
print("="*80)
print("\nData Types:")
print(df.dtypes)

print("\nBasic Statistics:")
print(df.describe())

In [ ]:
# Missing values analysis
print("Missing Values:")
print(df.isnull().sum())

# Distribution of delay target
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['delay_days'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Delay (days)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Actual Delays')
axes[0].grid(True, alpha=0.3)

axes[1].bar(['On-time', 'Delayed'], 
            [np.sum(df['delay_days'] <= 0), np.sum(df['delay_days'] > 0)],
            color=['green', 'red'], alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Count')
axes[1].set_title('On-time vs Delayed Shipments')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nDelay Summary:")
print(f"  Mean delay: {df['delay_days'].mean():.2f} days")
print(f"  Median delay: {df['delay_days'].median():.2f} days")
print(f"  Max delay: {df['delay_days'].max():.2f} days")
print(f"  % Delayed: {np.sum(df['delay_days'] > 0) / len(df) * 100:.1f}%")

In [ ]:
# Analyze delays by key categories
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# By product category
df.groupby('product_category')['delay_days'].mean().sort_values(ascending=False).plot(
    kind='bar', ax=axes[0, 0], color='steelblue', edgecolor='black')
axes[0, 0].set_title('Average Delay by Product Category')
axes[0, 0].set_ylabel('Delay (days)')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# By transportation mode
df.groupby('transportation_mode')['delay_days'].mean().sort_values(ascending=False).plot(
    kind='bar', ax=axes[0, 1], color='coral', edgecolor='black')
axes[0, 1].set_title('Average Delay by Transportation Mode')
axes[0, 1].set_ylabel('Delay (days)')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# By weather condition
df.groupby('weather_condition')['delay_days'].mean().sort_values(ascending=False).plot(
    kind='bar', ax=axes[1, 0], color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Average Delay by Weather Condition')
axes[1, 0].set_ylabel('Delay (days)')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Correlation with delay
correlation_data = df[['order_quantity', 'order_value', 'supplier_reliability_score',
                       'distance_km', 'fuel_price_index', 'port_congestion_score',
                       'customs_clearance_hours', 'scheduled_delivery_days',
                       'historical_delay_rate', 'supplier_inventory_level', 'delay_days']].corr()['delay_days'].sort_values(ascending=False)

correlation_data.drop('delay_days').plot(kind='barh', ax=axes[1, 1], color='purple', edgecolor='black')
axes[1, 1].set_title('Feature Correlation with Delay')
axes[1, 1].set_xlabel('Correlation Coefficient')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 3. Data Preprocessing and Feature Engineering

Handle categorical variables, normalize numerical features, and create interaction features.

In [ ]:
# Initialize feature engineer
engineer = FeatureEngineer()

# Add temporal features
df_processed = engineer.add_temporal_features(df.copy())

# Add interaction features
df_processed = engineer.add_interaction_features(df_processed)

print("Features before engineering:", len(df.columns))
print("Features after engineering:", len(df_processed.columns))

print("\nNew features created:")
new_features = set(df_processed.columns) - set(df.columns)
for feature in sorted(new_features):
    print(f"  - {feature}")

# Show sample of engineered features
print("\nSample of engineered data:")
print(df_processed[['date', 'day_of_week', 'month', 'is_weekend', 'estimated_transit_hours', 
                     'value_per_unit', 'reliability_consistency']].head())

In [ ]:
# Transform features (encoding and scaling)
X, feature_names = engineer.fit_transform(df_processed)
y = df_processed['delay_days'].values

print(f"Feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nNumber of features: {len(feature_names)}")
print("\nFeature names:")
for i, name in enumerate(feature_names, 1):
    print(f"  {i}. {name}")

# Display feature statistics after scaling
X_df = pd.DataFrame(X, columns=feature_names)
print("\nFeature statistics after scaling (first 10 features):")
print(X_df.iloc[:, :10].describe())

## 4. Train-Test Split

Split the dataset into training and testing sets using temporal ordering.

In [ ]:
# Split data temporally (sorted by date)
df_sorted = df_processed.sort_values('date').reset_index(drop=True)
X_sorted = X[df_sorted.index]
y_sorted = y[df_sorted.index]

# Temporal split: 70% train, 15% validation, 15% test
n_total = len(X_sorted)
n_train = int(0.70 * n_total)
n_val = int(0.15 * n_total)

X_train = X_sorted[:n_train]
y_train = y_sorted[:n_train]

X_val = X_sorted[n_train:n_train + n_val]
y_val = y_sorted[n_train:n_train + n_val]

X_test = X_sorted[n_train + n_val:]
y_test = y_sorted[n_train + n_val:]

print(f"Training set size: {len(X_train)} ({len(X_train)/n_total*100:.1f}%)")
print(f"Validation set size: {len(X_val)} ({len(X_val)/n_total*100:.1f}%)")
print(f"Test set size: {len(X_test)} ({len(X_test)/n_total*100:.1f}%)")

print(f"\nTraining target statistics:")
print(f"  Mean: {y_train.mean():.2f} days")
print(f"  Std: {y_train.std():.2f} days")
print(f"  % Delayed: {np.sum(y_train > 0) / len(y_train) * 100:.1f}%")

## 5. Build Baseline Model

Create a simple baseline model to establish a performance benchmark.

In [ ]:
# Train baseline linear regression model
print("Training baseline model (Linear Regression)...\n")

baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

# Predictions
y_train_pred_baseline = baseline_model.predict(X_train)
y_val_pred_baseline = baseline_model.predict(X_val)
y_test_pred_baseline = baseline_model.predict(X_test)

# Evaluate baseline
evaluator = ModelEvaluator()

train_metrics_baseline = evaluator.calculate_metrics(y_train, y_train_pred_baseline)
val_metrics_baseline = evaluator.calculate_metrics(y_val, y_val_pred_baseline)
test_metrics_baseline = evaluator.calculate_metrics(y_test, y_test_pred_baseline)

print("BASELINE MODEL PERFORMANCE")
print("="*60)
print("\nTraining Metrics:")
for key, value in train_metrics_baseline.items():
    print(f"  {key.upper()}: {value:.4f}")

print("\nValidation Metrics:")
for key, value in val_metrics_baseline.items():
    print(f"  {key.upper()}: {value:.4f}")

print("\nTest Metrics:")
for key, value in test_metrics_baseline.items():
    print(f"  {key.upper()}: {value:.4f}")

## 6. Train Advanced Models

Train multiple advanced machine learning models for comparison.

In [ ]:
# Train individual models
print("Training advanced models...\n")

models_dict = {
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.05, random_state=42, verbose=0),
    'LightGBM': LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.05, random_state=42, verbose=-1),
}

trained_models = {}
model_results = {}

for model_name, model in models_dict.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    
    # Predictions on validation set
    y_val_pred = model.predict(X_val)
    
    # Evaluate
    metrics = evaluator.calculate_metrics(y_val, y_val_pred)
    model_results[model_name] = metrics
    
    print(f"  ✓ Complete - Val R²: {metrics['r2']:.4f}, Val RMSE: {metrics['rmse']:.4f}\n")

print("All models trained successfully!")

In [ ]:
# Train ensemble model
print("Training Ensemble Model...")

ensemble = EnsembleDelayPredictor(['xgboost', 'lightgbm', 'random_forest'])
ensemble.train(X_train, y_train, X_val, y_val)

y_val_pred_ensemble = ensemble.predict(X_val)
metrics_ensemble = evaluator.calculate_metrics(y_val, y_val_pred_ensemble)
model_results['Ensemble'] = metrics_ensemble

print(f"  ✓ Complete - Val R²: {metrics_ensemble['r2']:.4f}, Val RMSE: {metrics_ensemble['rmse']:.4f}\n")

# Store best model
best_model_name = max(model_results, key=lambda x: model_results[x]['r2'])
print(f"\nBest model on validation set: {best_model_name}")
print(f"  R² Score: {model_results[best_model_name]['r2']:.4f}")
print(f"  RMSE: {model_results[best_model_name]['rmse']:.4f}")

## 7. Model Evaluation and Comparison

Evaluate all models using comprehensive metrics and compare their performance.

In [ ]:
# Evaluate on test set
print("COMPREHENSIVE MODEL EVALUATION ON TEST SET")
print("="*80)

test_results = {}

# Baseline model
y_test_pred_baseline = baseline_model.predict(X_test)
test_results['Linear Regression (Baseline)'] = evaluator.calculate_metrics(y_test, y_test_pred_baseline)

# Individual models
for model_name, model in trained_models.items():
    y_test_pred = model.predict(X_test)
    test_results[model_name] = evaluator.calculate_metrics(y_test, y_test_pred)

# Ensemble
y_test_pred_ensemble = ensemble.predict(X_test)
test_results['Ensemble'] = evaluator.calculate_metrics(y_test, y_test_pred_ensemble)

# Create comparison dataframe
results_df = pd.DataFrame(test_results).T
results_df = results_df.round(4)

print("\nTest Set Results:")
print(results_df.sort_values('r2', ascending=False))

# Find best model
best_model = results_df['r2'].idxmax()
print(f"\n✓ Best Model: {best_model}")
print(f"  R² Score: {results_df.loc[best_model, 'r2']:.4f}")
print(f"  RMSE: {results_df.loc[best_model, 'rmse']:.4f}")
print(f"  MAE: {results_df.loc[best_model, 'mae']:.4f}")

In [ ]:
# Visualize model comparison
PredictionVisualizer.plot_metrics_comparison(test_results)

In [ ]:
# Detailed analysis of best model (Ensemble)
print("\nDETAILED ANALYSIS OF BEST MODEL (ENSEMBLE)")
print("="*80)

# Classification metrics (delayed vs on-time)
class_metrics = evaluator.calculate_classification_metrics(y_test, y_test_pred_ensemble, threshold=0.5)

print("\nClassification Metrics (Delayed vs On-time):")
for key, value in class_metrics.items():
    print(f"  {key}: {value:.4f}")

# Residual analysis
residual_stats = evaluator.get_residual_statistics(y_test, y_test_pred_ensemble)

print("\nResidual Statistics:")
for key, value in residual_stats.items():
    print(f"  {key}: {value:.4f}")

# Visualizations
print("\nGenerating visualizations...")
PredictionVisualizer.plot_predictions_vs_actual(y_test, y_test_pred_ensemble, "Ensemble Model: Actual vs Predicted Delays")
PredictionVisualizer.plot_error_distribution(y_test, y_test_pred_ensemble, "Ensemble Model: Prediction Error Distribution")

## 8. Feature Importance Analysis

Analyze which features have the greatest impact on delay predictions.

In [ ]:
# Extract feature importance from different models
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# XGBoost
xgb_model = trained_models['XGBoost']
xgb_importance = xgb_model.feature_importances_
top_indices = np.argsort(xgb_importance)[-15:]
axes[0, 0].barh(range(len(top_indices)), xgb_importance[top_indices], color='steelblue', edgecolor='black')
axes[0, 0].set_yticks(range(len(top_indices)))
axes[0, 0].set_yticklabels([feature_names[i] for i in top_indices])
axes[0, 0].set_xlabel('Importance')
axes[0, 0].set_title('XGBoost - Top 15 Features')
axes[0, 0].invert_yaxis()

# LightGBM
lgb_model = trained_models['LightGBM']
lgb_importance = lgb_model.feature_importances_
top_indices = np.argsort(lgb_importance)[-15:]
axes[0, 1].barh(range(len(top_indices)), lgb_importance[top_indices], color='coral', edgecolor='black')
axes[0, 1].set_yticks(range(len(top_indices)))
axes[0, 1].set_yticklabels([feature_names[i] for i in top_indices])
axes[0, 1].set_xlabel('Importance')
axes[0, 1].set_title('LightGBM - Top 15 Features')
axes[0, 1].invert_yaxis()

# Random Forest
rf_model = trained_models['Random Forest']
rf_importance = rf_model.feature_importances_
top_indices = np.argsort(rf_importance)[-15:]
axes[1, 0].barh(range(len(top_indices)), rf_importance[top_indices], color='lightgreen', edgecolor='black')
axes[1, 0].set_yticks(range(len(top_indices)))
axes[1, 0].set_yticklabels([feature_names[i] for i in top_indices])
axes[1, 0].set_xlabel('Importance')
axes[1, 0].set_title('Random Forest - Top 15 Features')
axes[1, 0].invert_yaxis()

# Average importance across models
avg_importance = (np.array(xgb_importance) + np.array(lgb_importance) + np.array(rf_importance)) / 3
top_indices = np.argsort(avg_importance)[-15:]
axes[1, 1].barh(range(len(top_indices)), avg_importance[top_indices], color='gold', edgecolor='black')
axes[1, 1].set_yticks(range(len(top_indices)))
axes[1, 1].set_yticklabels([feature_names[i] for i in top_indices])
axes[1, 1].set_xlabel('Importance')
axes[1, 1].set_title('Average Importance - Top 15 Features')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()

# Print top 10 features
print("\nTop 10 Most Important Features (by average):")
sorted_indices = np.argsort(avg_importance)[::-1]
for rank, idx in enumerate(sorted_indices[:10], 1):
    print(f"  {rank}. {feature_names[idx]}: {avg_importance[idx]:.4f}")

## 9. Make Predictions on New Data

Use the best-performing model to make predictions and generate actionable insights.

In [ ]:
# Make predictions on entire test set
predictions = ensemble.predict(X_test)
uncertainty = ensemble.predict_with_uncertainty(X_test)[1]

# Create results dataframe
test_data = df_sorted[n_train + n_val:].reset_index(drop=True)
results_df = test_data.copy()
results_df['predicted_delay_days'] = predictions
results_df['uncertainty'] = uncertainty
results_df['confidence_lower'] = predictions - 1.96 * uncertainty
results_df['confidence_upper'] = predictions + 1.96 * uncertainty
results_df['is_delayed'] = (predictions > 0).astype(int)
results_df['risk_level'] = pd.cut(
    predictions,
    bins=[0, 1, 3, 5, float('inf')],
    labels=['Low', 'Medium', 'High', 'Critical']
)

print("PREDICTIONS ON TEST SET")
print("="*80)
print(f"\nTotal test shipments: {len(results_df)}")

# Risk distribution
print("\nRisk Level Distribution:")
risk_counts = results_df['risk_level'].value_counts()
for risk in ['Low', 'Medium', 'High', 'Critical']:
    count = risk_counts.get(risk, 0)
    pct = count / len(results_df) * 100
    print(f"  {risk}: {count} ({pct:.1f}%)")

# Show high-risk shipments
analyzer = DelayAnalyzer()
high_risk = analyzer.identify_high_risk_shipments(test_data, predictions, threshold=5.0)

print(f"\nHigh-Risk Shipments (predicted delay > 5 days): {len(high_risk)}")
print("\nTop 10 High-Risk Shipments:")
high_risk_display = high_risk[['supplier_id', 'warehouse_id', 'product_category', 
                               'transportation_mode', 'predicted_delay', 'risk_level']].head(10)
print(high_risk_display.to_string())

In [ ]:
# Actionable insights
print("\nACTIONABLE INSIGHTS FROM PREDICTIONS")
print("="*80)

# Analysis by category
print("\nAverage Predicted Delay by Product Category:")
category_delays = results_df.groupby('product_category')['predicted_delay_days'].agg(['mean', 'count'])
print(category_delays.round(2))

# Analysis by transportation mode
print("\nAverage Predicted Delay by Transportation Mode:")
transport_delays = results_df.groupby('transportation_mode')['predicted_delay_days'].agg(['mean', 'count'])
print(transport_delays.round(2))

# Most problematic suppliers
print("\nTop 5 Suppliers with Highest Average Predicted Delays:")
supplier_delays = results_df.groupby('supplier_id')['predicted_delay_days'].mean().sort_values(ascending=False).head(5)
for supplier_id, delay in supplier_delays.items():
    print(f"  Supplier {supplier_id}: {delay:.2f} days average delay")

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# By category
results_df.boxplot(column='predicted_delay_days', by='product_category', ax=axes[0, 0])
axes[0, 0].set_title('Predicted Delays by Product Category')
axes[0, 0].set_xlabel('Category')
axes[0, 0].set_ylabel('Delay (days)')

# By transportation
results_df.boxplot(column='predicted_delay_days', by='transportation_mode', ax=axes[0, 1])
axes[0, 1].set_title('Predicted Delays by Transportation Mode')
axes[0, 1].set_xlabel('Mode')
axes[0, 1].set_ylabel('Delay (days)')

# Risk distribution
risk_dist = results_df['risk_level'].value_counts()
axes[1, 0].bar(risk_dist.index, risk_dist.values, color=['green', 'yellow', 'orange', 'red'], edgecolor='black')
axes[1, 0].set_title('Distribution of Risk Levels')
axes[1, 0].set_ylabel('Count')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Delay distribution
axes[1, 1].hist(results_df['predicted_delay_days'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[1, 1].axvline(results_df['predicted_delay_days'].mean(), color='r', linestyle='--', linewidth=2, label=f'Mean: {results_df["predicted_delay_days"].mean():.2f}')
axes[1, 1].set_title('Distribution of Predicted Delays')
axes[1, 1].set_xlabel('Delay (days)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Model Deployment Preparation

Save the model and prepare it for production deployment.

In [ ]:
import joblib

# Save the ensemble model
model_path = '/workspaces/project-defense/supply_chain_prediction/ensemble_model.pkl'
engineer_path = '/workspaces/project-defense/supply_chain_prediction/feature_engineer.pkl'

joblib.dump(ensemble, model_path)
joblib.dump(engineer, engineer_path)

print("MODEL DEPLOYMENT PREPARATION")
print("="*80)
print(f"\n✓ Ensemble model saved to: {model_path}")
print(f"✓ Feature engineer saved to: {engineer_path}")

# Create a deployment summary
deployment_summary = {
    'model_type': 'Ensemble (XGBoost, LightGBM, Random Forest)',
    'training_samples': len(X_train),
    'validation_samples': len(X_val),
    'test_samples': len(X_test),
    'features': feature_names,
    'performance_metrics': {
        'test_r2_score': float(test_results['Ensemble']['r2']),
        'test_rmse': float(test_results['Ensemble']['rmse']),
        'test_mae': float(test_results['Ensemble']['mae']),
        'test_mape': float(test_results['Ensemble']['mape']),
    },
    'data_info': {
        'total_features': len(feature_names),
        'numerical_features': len(engineer.numerical_features),
        'categorical_features': len(engineer.categorical_features),
    },
    'model_assumptions': [
        'Temporal data split used (no data leakage)',
        'Features scaled using StandardScaler',
        'Categorical variables label-encoded',
        'Missing values handled during data generation',
    ],
    'deployment_notes': [
        'Model requires feature engineer for transformation',
        'Input features must match training features in order',
        'For optimal performance, ensure data quality matches training distribution',
        'Regular monitoring recommended for data drift',
        'Retrain model quarterly or when performance degrades',
    ]
}

import json

summary_path = '/workspaces/project-defense/supply_chain_prediction/model_deployment_summary.json'
with open(summary_path, 'w') as f:
    json.dump(deployment_summary, f, indent=2)

print(f"✓ Deployment summary saved to: {summary_path}")

print("\nDEPLOYMENT SUMMARY")
print("-"*80)
print(f"Model Type: {deployment_summary['model_type']}")
print(f"\nPerformance Metrics (Test Set):")
for metric, value in deployment_summary['performance_metrics'].items():
    print(f"  {metric}: {value:.4f}")

print(f"\nData Configuration:")
for key, value in deployment_summary['data_info'].items():
    print(f"  {key}: {value}")

print(f"\nModel Assumptions:")
for assumption in deployment_summary['model_assumptions']:
    print(f"  • {assumption}")

print(f"\nDeployment Notes:")
for note in deployment_summary['deployment_notes']:
    print(f"  • {note}")

In [ ]:
print("\n" + "="*80)
print("SUPPLY CHAIN DELAY PREDICTION SYSTEM - COMPLETE")
print("="*80)

print("\nSUMMARY OF RESULTS:")
print(f"  • Best Model: Ensemble (XGBoost + LightGBM + Random Forest)")
print(f"  • Test R² Score: {test_results['Ensemble']['r2']:.4f}")
print(f"  • Test RMSE: {test_results['Ensemble']['rmse']:.4f} days")
print(f"  • Test MAE: {test_results['Ensemble']['mae']:.4f} days")

print(f"\nPREDICTION STATISTICS:")
print(f"  • Total Test Shipments: {len(results_df)}")
print(f"  • High-Risk Shipments: {len(high_risk)}")
print(f"  • Average Predicted Delay: {predictions.mean():.2f} days")
print(f"  • Max Predicted Delay: {predictions.max():.2f} days")

print(f"\nMODELS TRAINED: {len(models_dict) + 1}")
for model_name in test_results.keys():
    r2 = test_results[model_name]['r2']
    print(f"  • {model_name}: R² = {r2:.4f}")

print(f"\nFEATURES ENGINEERED: {len(feature_names)}")

print(f"\nMODEL ARTIFACTS SAVED:")
print(f"  • Model: {model_path}")
print(f"  • Feature Engineer: {engineer_path}")
print(f"  • Deployment Summary: {summary_path}")

print("\nNEXT STEPS:")
print("  1. Monitor model performance in production")
print("  2. Set up automated retraining pipeline")
print("  3. Establish alerts for high-risk shipments")
print("  4. Integrate with supply chain management system")
print("  5. Collect feedback and improve model periodically")

print("\n" + "="*80)